In [1]:
# selenium으로 키를 조작하기 위한 import
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ETC
import time # to wait not to seem as "bot"
import csv

4.22.0


# 3. 역대 대통령 데이터
https://www.pa.go.kr/research/contents/speech/index.jsp

### 대통령별 체크박스 Xpath

In [2]:
# 전체 : //*[@id="prstNm0"]
# 이승만 : //*[@id="prstNm1"]
# 윤보선 : //*[@id="prstNm2"]
# 박정희 : //*[@id="prstNm3"]
# 최규하 : //*[@id="prstNm4"]
# 전두환 : //*[@id="prstNm5"]
# 노태우 : //*[@id="prstNm6"]
# 김영삼 : //*[@id="prstNm7"]
# 김대중 : //*[@id="prstNm8"]
# 노무현 : //*[@id="prstNm9"]
# 이명박 : //*[@id="prstNm10"]
# 박근혜 : //*[@id="prstNm11"]
# 문재인 : //*[@id="prstNm12"]

In [14]:
# search speech data of specific president
def search(president):
    global driver
    name_d = {"전체":0, "이승만":1, "윤보선":2, "박정희":3, "최규하":4, "전두환":5, 
              "노태우":6, "김영삼":7, "김대중":8, "노무현":9, "이명박":10, "박근혜":11, 
              "문재인":12}
    presid_num = name_d[f"{president}"]

    checkbox = driver.find_element(By.XPATH, f'//*[@id="prstNm{presid_num}"]')
    search = driver.find_element(By.XPATH, '//*[@id="frm"]/fieldset[1]/div[2]/div[5]/button[2]')
    checkbox.click()
    time.sleep(1)
    search.click()
    time.sleep(1)
    doc_button = driver.find_element(By.XPATH, '//*[@id="m-items2"]/li[2]/a')
    doc_button.click()

In [15]:
# Helper function to click page button
def click_section_button():
    global driver
    try:
        page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[13]/a')))
        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(2)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
        time.sleep(2)
    except TimeoutException:
        print(f"Section button not found")
        raise  # if TimeoutException : throw Exception & quit

In [16]:
# Helper function to click page button
def click_page_button(page_num):
    global driver
    try:
        if page_num == 1:
            page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[{page_num+2}]/strong')))
        else:
            page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[{page_num+2}]/a')))
        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
    except TimeoutException:
        print(f"Page button {page_num} not found")
        raise  # if TimeoutException : throw Exception & quit

In [17]:
# 10개 페이지들

driver = webdriver.Chrome()
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "문재인"
#str(input("President's name you want : "))
search(president)

for page_num in range(1, 11):
    click_page_button(page_num)
    try:
        for title_num in range(1, 21):
            try:
                clik = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
                driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
                time.sleep(1)
                driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

                o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
                o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
                o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
                o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

                title = o_title.text
                type = o_type.text
                date = o_date.text
                speech = o_speech.text
                speaker = president

                d = {"index": idx, "title": title, "type": type, "date": date, "speaker": president, "speech": speech}
                speech_data.append(d)
                print(f"{idx}. {title}")
                idx += 1

                driver.back()
            except Exception as e:
                print(f"ERROR OCCURED {str(e)}")
    except Exception as e:
        print(f"ERROR OCCURED {str(e)}")
driver.quit()

1. 제19대 대통령 취임식
2. 비정규직 근로자와의 대화
3. 미세먼지 바로알기 교실 현장 방문
4. 국방부 순시
5. 합동참모본부 순시
6. 제37주년 5·18민주화운동 기념식
7. 노무현 대통령 서거 8주기 추도식
8. 수석·보좌관회의
9. 수석·보좌관회의
10. 제22회 바다의 날 기념식
11. 수석·보좌관회의
12. OECD 국제교통포럼(ITF) 사무총장 당선 축전
13. 치매국가책임제 SNS 메시지
14. 치매국가책임제 관련 현장 방문
15. 제62회 현충일 추념식
16. 용산소방서 방문 후 SNS 메시지
17. 현장 소방대원들과의 대화
18. 국가안전보장회의(NSC)
19. 제30주년 6·10민주항쟁 기념식
20. 2017년도 추가경정예산안 관련 국회 시정 연설
21. 한미연합사령부 순시
22. 제19회 미주한인체전 축전
23. 제1회 전국 시·도지사 간담회
24. 6·15남북공동선언 17주년 기념식
25. 수석·보좌관회의
26. 아시아인프라투자은행(AIIB) 연차총회 개회식
27. 원자력발전소 고리1호기 영구 정지 선포식


KeyboardInterrupt: 

In [12]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome()
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "문재인"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 70):
    click_page_button(page_num)
    if page_num % 10 ==0 :
        click_section_button(page_num)
    
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            type = o_type.text
            date = o_date.text
            speech = o_speech.textS
            speaker = president

            d = {"index": idx, "title": title, "type": type, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
driver.quit()

ERROR OCCURED 'WebElement' object has no attribute 'textS'
Page button 2 not found


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A2F2EEA2+31554]
	(No symbol) [0x00007FF7A2EA7ED9]
	(No symbol) [0x00007FF7A2D6872A]
	(No symbol) [0x00007FF7A2DB8434]
	(No symbol) [0x00007FF7A2DB853C]
	(No symbol) [0x00007FF7A2DFF6A7]
	(No symbol) [0x00007FF7A2DDD06F]
	(No symbol) [0x00007FF7A2DFC977]
	(No symbol) [0x00007FF7A2DDCDD3]
	(No symbol) [0x00007FF7A2DAA33B]
	(No symbol) [0x00007FF7A2DAAED1]
	GetHandleVerifier [0x00007FF7A3238B1D+3217341]
	GetHandleVerifier [0x00007FF7A3285AE3+3532675]
	GetHandleVerifier [0x00007FF7A327B0E0+3489152]
	GetHandleVerifier [0x00007FF7A2FDE776+750614]
	(No symbol) [0x00007FF7A2EB375F]
	(No symbol) [0x00007FF7A2EAEB14]
	(No symbol) [0x00007FF7A2EAECA2]
	(No symbol) [0x00007FF7A2E9E16F]
	BaseThreadInitThunk [0x00007FFAFFC9257D+29]
	RtlUserThreadStart [0x00007FFB008CAF28+40]
